In [ ]:
# If you see 'Missing dependencies for SOCKS support', install PySocks%pip install pysocks requests[socks]

In [9]:
#!/usr/bin/env python3
"""
Debug script: direct LIMIT BUY order to LATOKEN API at price 0.001 USD for 5 TDG units.
"""
import os
import time
import json
import hmac
import hashlib
import requests

from dotenv import load_dotenv
load_dotenv()


# Load credentials and trading pair IDs from environment
api_key     = os.getenv("LATOKEN_API_KEY")
api_secret  = os.getenv("LATOKEN_API_SECRET")
base_url    = os.getenv("LATOKEN_BASE_URL", "https://api.latoken.com")
currency_id = os.getenv("LATOKEN_CURRENCY_ID")
quote_id    = os.getenv("LATOKEN_QUOTE_ID")

In [10]:
print(api_key)

b135be25-3040-44db-bd81-38f2e2ea5914


In [14]:
# Configure proxy (set LATOKEN_PROXY=socks5://127.0.0.1:9999)
proxy_url = os.getenv("LATOKEN_PROXY", "socks5://127.0.0.1:9999")
session = requests.Session()
session.proxies.update({"http": proxy_url, "https": proxy_url})
print("Using proxy:", session.proxies)

# Debug order parameters
purchase_price = 0.001  # USD per TDG
quantity       = 5.0    # TDG to buy

# Construct payload for limit BUY order
payload = {
    "baseCurrency":  currency_id,
    "quoteCurrency": quote_id,
    "side":          "BUY",
    "type":          "LIMIT",
    "condition":     "GTC",
    "quantity":      str(quantity),
    "price":         str(purchase_price),
    "timestamp":     int(time.time()),
}
body = json.dumps(payload, separators=(",", ":"), ensure_ascii=False)

# Sign request with HMAC-SHA512
signature = hmac.new(
    api_secret.encode("utf-8"),
    body.encode("utf-8"),
    hashlib.sha512
).hexdigest()
headers = {
    "X-LA-APIKEY":    api_key,
    "X-LA-SIGNATURE": signature,
    "Content-Type":   "application/json",
}

# Send HTTP POST to /v2/order
url = f"{base_url}/v2/order"
print("POST →", url)
print("Headers:", headers)
print("Body:\n", body)
resp = session.post(url, headers=headers, data=body)
print("Status code:", resp.status_code)
try:
    print("Response JSON:\n", resp.json())
except ValueError:
    print("Response text:\n", resp.text)

TypeError: key: expected bytes or bytearray, but got 'str'